In [1]:
!pip install lingam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 855.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.7/95.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 93.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 15.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 16.5 MB/s eta 0:00:00
  Created wheel for semopy: filename=semopy-2.3.11-py3-none-any.whl size=1659681 sha256=ec38813796755bfcdf3a1151060c

In [1]:
#German
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split

from csse import CSSE
from prepare_dataset import *

import pickle

import numpy as np
import pandas as pd
import graphviz
import lingam
from lingam.utils import print_causal_directions, print_dagc, make_dot

from tqdm import tqdm
import random as rnd

from IPython.display import display

import warnings

warnings.filterwarnings('ignore')

In [2]:
#German
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split

from csse import CSSE
from prepare_dataset import *

import pickle

import numpy as np
import pandas as pd
import graphviz
import lingam
from lingam.utils import print_causal_directions, print_dagc, make_dot

from tqdm import tqdm
import random as rnd

from IPython.display import display

import warnings

warnings.filterwarnings('ignore')



class CCSSE:
    def __init__(self, dataset, samples = None, K = 5, generation = 10):
        self.dataset = dataset
        self.samples = samples
        self.K = K
        self.generation = generation
        
        self.x_train, self.x_test, self.y_train, self.y_test, self.dfx_full, self.dfy_full = self.get_datasets_train_test()

        self.bb_model, self.p = self.get_bb_model()
        self.explainerCSSE = self.get_model_contrafactual()
        self.model_causal, self.df_causal_effects, self.df_error, self.causal_order = self.get_model_causality()
    
        self.run_dict = {}
        self.run_non_causal_dict = {}
    
    def get_datasets_train_test(self):
        if self.dataset == "german_short":
            map_columns = {
                'Unnamed: 0': 'index',
                'x0': 'Sex',
                'x1': 'Age',
                'x2': 'Credit',
                'x3': 'LoanDuration',
            }
            x_train = pd.read_csv("data/algrec_german/X_train_short.csv").rename(columns = map_columns)
            x_test = pd.read_csv("data/algrec_german/X_test_short.csv").rename(columns = map_columns)
            y_train = pd.read_csv("data/algrec_german/y_train_short.csv").rename(columns={'Unnamed: 0': 'index'})
            y_test = pd.read_csv("data/algrec_german/y_test_short.csv").rename(columns={'Unnamed: 0': 'index'})
            x_train = x_train.set_index('index')
            x_test = x_test.set_index('index')
            y_train = y_train.set_index('index')
            y_test = y_test.set_index('index')
            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])

        elif self.dataset == "german_medium":
            map_columns = {
                'Unnamed: 0': 'index',
                'x0': 'Sex',
                'x1': 'Age',
                'x2': 'Credit',
                'x3': 'LoanDuration',
                'x4': 'CheckingAccountBalance',
                'x5':'SavingsAccountBalance',
                'x6':'HousingStatus'
            }
            x_train = pd.read_csv("data/algrec_german/X_train.csv").rename(columns = map_columns)
            x_test = pd.read_csv("data/algrec_german/X_test.csv").rename(columns = map_columns)
            y_train = pd.read_csv("data/algrec_german/y_train.csv").rename(columns={'Unnamed: 0': 'index'})
            y_test = pd.read_csv("data/algrec_german/y_test.csv").rename(columns={'Unnamed: 0': 'index'})
            x_train = x_train.set_index('index')
            x_test = x_test.set_index('index')
            y_train = y_train.set_index('index')
            y_test = y_test.set_index('index')
            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])

        elif self.dataset == "german_full":
            df_main = prepare_german_dataset("german_credit.csv", "data/")
            columns = df_main.columns
            class_name = 'default' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'np':
            df_main = pd.read_csv("data/breast_coimbra_np.csv")
            columns = df_main.columns
            class_name = 'Classification' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'nm':
            df_main = pd.read_csv("data/heloc_dataset_v1_nm.csv")
            columns = df_main.columns
            class_name = 'RiskPerformance' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'cm':
            df_main = pd.read_csv("data/house_votes_84_processada_cm.csv")
            columns = df_main.columns
            class_name = 'Class Name' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'ng':
            df_main = pd.read_csv("data/ionosphere_ng.csv")
            columns = df_main.columns
            class_name = 'target' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'tokyo_ng':
            df_main = pd.read_csv("data/Tokyo_ng.csv").rename(columns = {'Unnamed: 44': 'class'})
            columns = df_main.columns
            class_name = 'class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'breast-cancer_ng':
            df_main = pd.read_csv("data/breast-cancer_ng.csv")
            columns = df_main.columns
            class_name = 'diagnosis' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
        
        elif self.dataset == 'cp':
            df_train = pd.read_csv("data/monks-1_train_cp.csv")

            #Get the input features
            columns = df_train.columns
            class_name = 'Class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)
            columns_tmp.remove('Id')

            x_train = df_train[columns_tmp]
            y_train = df_train[['Class']]

            # x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            model = RandomForestClassifier()  
            model.fit(x_train, y_train)

            df_test = pd.read_csv('data/monks-1_test_cp.csv')

            #Get the input features
            columns = df_test.columns
            class_name = 'Class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)
            columns_tmp.remove('Id')

            x_test = df_test[columns_tmp]
            y_test = df_test[['Class']]

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'cg':
            df_main = pd.read_csv("data/mushrooms_processada_cg.csv")[:1000]
            columns = df_main.columns
            class_name = 'class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'Phishing_cg':
            df_main = pd.read_csv("data/Phishing_cg.csv")[2:1000].astype(float)
            columns = df_main.columns
            class_name = 'Class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])

        else:
            x_train = pd.DataFrame()
            x_test = pd.DataFrame()
            y_train = pd.DataFrame()
            y_test = pd.DataFrame()
            dfx_full = pd.DataFrame()
            dfy_full = pd.DataFrame()

        return x_train, x_test, y_train, y_test, dfx_full, dfy_full
    

    def get_bb_model(self):
        bb_model = RandomForestClassifier()  
        bb_model.fit(self.x_train, self.y_train)

        p = bb_model.predict(self.x_test)

        print(classification_report(self.y_test, p))

        return bb_model, p

    def get_model_contrafactual(self):
        return CSSE(self.dfx_full, self.bb_model, K = self.K, num_gen = self.generation)

    def get_model_causality(self):
        model_causal = lingam.DirectLiNGAM()
        model_causal.fit(self.dfx_full)
            
        labels = [f'{i}' for i in self.dfx_full.columns]
        causal_order = [labels[x] for x in model_causal.causal_order_]
        
        matrix = model_causal.adjacency_matrix_
        from_list = []
        to_list = []
        effect_list = []

        # Iteração sobre a matriz para extrair os valores e suas posições
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                if matrix[i][j] != 0:
                    from_list.append(j)
                    to_list.append(i)
                    effect_list.append(matrix[i][j])

        # Criando o DataFrame
        df_causal_effects = pd.DataFrame({'from': from_list, 'to': to_list, 'effect': effect_list})
        labels = [f'{i}' for i in self.dfx_full.columns]
        df_causal_effects['from'] = df_causal_effects['from'].apply(lambda x : labels[x])
        df_causal_effects['to'] = df_causal_effects['to'].apply(lambda x : labels[x])


        matrix_error = model_causal.get_error_independence_p_values(self.dfx_full)
        from_list = []
        to_list = []
        effect_list = []

        # Iteração sobre a matriz para extrair os valores e suas posições
        for i in range(len(matrix_error)):
            for j in range(i + 1, len(matrix_error[i])):
                if matrix_error[i][j] != 0:
                    from_list.append(j)
                    to_list.append(i)
                    effect_list.append(matrix_error[i][j])

        # Criando o DataFrame
        df_error = pd.DataFrame({'from': from_list, 'to': to_list, 'effect': effect_list})
        labels = [f'{i}' for i in self.dfx_full.columns]
        df_error['from'] = df_error['from'].apply(lambda x : labels[x])
        df_error['to'] = df_error['to'].apply(lambda x : labels[x])
        

        return model_causal, df_causal_effects, df_error, causal_order
        
    
    def print_causal_graph(self):
        make_dot(self.model_causal.adjacency_matrix_)

    def run_non_causal(self):
        self.run_non_causal_dict = {}

        if isinstance(self.samples, list):
            self.create_run_dict(self)
            for sample in self.samples:
                self.run_non_causal_sample(sample)
                
        elif isinstance(self.samples, int):
            for sample in range(self.samples):
                self.run_non_causal_sample(sample)
        
        else: 
            for sample in range(10):
                self.run_non_causal_sample(sample)
                
    def run_non_causal_sample(self, sample):
        self.run_non_causal_dict[sample] = {}
        original_instance = self.x_test.iloc[sample].copy()
        contrafactual_set, solution = self.explainerCSSE.explain(original_instance, self.p[sample]) #Method returns the list of counterfactuals and the explanations generated from them

        self.run_non_causal_dict[sample]['solution'] = solution

    def run_causal(self):
        self.run_dict = {}
        self.run_dict['global_numbers'] = {
                    "global_quant_changes": 0,
                    "global_quant_causal_changes": 0,
                    "global_quant_causal_rules": 0,
                    "global_quant_zeros_causal": 0,
                    "global_quant_full_causal": 0,
                    "global_quant_causal_contrafac": 0,
                    "global_quant_maioria_causal_satisfeita": 0,
                    "global_quant_contrafac_unico": 0
            }
        self.global_quant_contrafac_max = 0
        if isinstance(self.samples, list):
            for sample in self.samples:
                self.run_causal_sample(sample)
                
        elif isinstance(self.samples, int):
            for sample in range(self.samples):
                self.run_causal_sample(sample)
        
        else: 
            for sample in range(10):
                self.run_causal_sample(sample)
        
        self.global_quant_contrafac_max = self.K * len(self.run_dict)


    def run_causal_sample(self, sample):
        if isinstance(self.samples, list):
            original_instance = self.dfx_full.iloc[sample]
        else:
            original_instance = self.x_test.iloc[sample]
        self.run_dict[sample] = {}
        self.run_dict[sample]['original_instance'] = original_instance

        print(f'Running original instance:\n {display(original_instance)}')

        causal_explain = self.get_causal_explain(sample)
        self.run_dict[sample]['causal_explain'] = causal_explain

        list_analyse = []
        for contrafactual in causal_explain[0]:
            list_analyse.append(self.analyse_contrafac(contrafactual, causal_explain[1], causal_explain[2]))

        self.run_dict[sample]['list_analyse'] = list_analyse
        self.analyse_explaination(sample)

    def analyse_contrafac(self, contrafac, df, original_ind):
        columns = [x.column for x in contrafac]
        condicao = (df['to'].isin(columns)) & (df['from'].isin(columns))
        ind = original_ind[columns]
        return [contrafac, df[condicao], ind]

    def get_causal_explain(self, sample):
        if isinstance(self.samples, list):
            original_ind = self.dfx_full.iloc[sample].copy()
        else:
            original_ind = self.x_test.iloc[sample].copy() #Original instance
        #self.ind_cur_class = ind_cur_class #Index in the shap corresponds to the original instance class
        self.explainerCSSE.current_class = self.p[sample] #Original instance class
        self.explainerCSSE.original_ind = original_ind
        
        ind_cur_class = self.explainerCSSE.getBadClass()

        #Gets the valid values range of each feature
        features_range = []
        features_range = self.explainerCSSE.getFeaturesRange()

        #The DataFrame df will have the current population
        df = pd.DataFrame(columns=self.explainerCSSE.input_dataset.columns)

        #Generates the initial population with popinitial mutants        
        self.explainerCSSE.getPopInicial(df, features_range)
        df_causal = df.copy()
        dict_dfs = {}

        # for g in tqdm(range(self.explainerCSSE.num_gen), desc= "Processing..."):
        for g in tqdm(range(self.generation), desc= "Processing..."):

            #To use on the parents of each generation
            old_parents = pd.DataFrame(columns=self.explainerCSSE.input_dataset.columns)

            #Copy parents to the next generation
            old_parents = df_causal.copy()
            dict_dfs[g] = {}

            parents_causal = self.apply_causality(old_parents)
            dict_dfs[g]['causal_parents'] = parents_causal
            #df will contain the new population
            df_causal = pd.DataFrame(columns=self.explainerCSSE.input_dataset.columns)
            evaluation_causal = []

            #Assessing generation counterfactuals
            self.explainerCSSE.fitness(dict_dfs[g]['causal_parents'], evaluation_causal, ind_cur_class)

            #The original individual will always be in the 0 position of the df - So that it is normalized too (it will be used later in the distance function)
            df_causal.loc[0] = original_ind.copy()

            #Copies to the next generation the per_elit best individuals
            self.explainerCSSE.elitism(evaluation_causal, df_causal, parents_causal)
            number_cross_repetitions = 0
            while len(df_causal) < self.explainerCSSE.pop_size + 1: #+1, as the 1st position is used to store the reference individual
                number_cross_repetitions_causal = self.explainerCSSE.crossover(df_causal, parents_causal, evaluation_causal, number_cross_repetitions)

                mutation_op = rnd.random()
                if mutation_op <= self.explainerCSSE.mutation_proba:
                    self.explainerCSSE.mutation(df_causal, len(df_causal) - 1, features_range)


        evaluation = []
        evaluation_causal = []

        #Evaluating the latest generation
        self.explainerCSSE.fitness(df_causal, evaluation_causal, ind_cur_class)

        #Order the last generation by distance to the original instance     
        evaluation_causal.sort(key=lambda individual: individual.aval_norm) 

        #Getting the counterfactual CAUSAL set
        contrafactual_set_causal, solution_list_causal = self.explainerCSSE.getContrafactual(df_causal, evaluation_causal) 

        dict_dfs['contrafactual_set_causal'] = contrafactual_set_causal
        dict_dfs['solution_list_causal'] = solution_list_causal
        
        df_contrafac_causal = self.get_contrafac_df_causal(solution_list_causal)
        return [solution_list_causal, df_contrafac_causal, original_ind]
    

    def apply_causality(self, df):
        df_apply_causal = pd.DataFrame(columns = df.columns)
        original = df.iloc[0]
        df_apply_causal.loc[0] = original
        for index, df_row in df.iloc[1:].iterrows():
            causal_ind = df_row.copy()
            for column in self.causal_order:
                value_diff = causal_ind[column] - original[column]
                if value_diff != 0:
                    tmp_effects = self.df_causal_effects[self.df_causal_effects['from'] == column]
                    for index, row in tmp_effects.iterrows():
    #                     prob = rnd.random()
    #                     if row['probability'] <= prob:
                        tmp_error = self.df_error[self.df_error['from'].isin([column, row['to']]) | self.df_error['to'].isin([column, row['to']])]
                        error_value = tmp_error['effect'].iloc[0]
    #                     print(f'error value = {error_value}')
                        causal_ind[row['to']] = causal_ind[row['to']] + (value_diff * row['effect']) + tmp_error['effect'].iloc[0]
            df_apply_causal.loc[len(df_apply_causal)] = causal_ind
        return df_apply_causal


    def get_contrafac_df_causal(self, solution_list_causal):
        lista_solution_causal = [[t.column for t in sublist] for sublist in solution_list_causal]

        # Inicializa uma lista para armazenar os resultados
        resultados = []

        # Loop sobre os valores na lista
        for lista_valores in lista_solution_causal:
            if len(lista_valores) > 1:
                for v1 in lista_valores:
                    for v2 in lista_valores:
                        if v1 != v2:
                            # Cria uma condição para cada par de valores diferentes na lista
                            condicao = (self.df_causal_effects['to'].isin([v1, v2])) & (self.df_causal_effects['from'].isin([v1, v2]))
                            # Realiza a busca no DataFrame usando a condição e armazena os resultados
                            resultados.append(self.df_causal_effects[condicao])

        # Concatena os resultados em um único DataFrame
        if resultados:
            resultado_final = pd.concat(resultados)
            resultado_final = resultado_final.drop_duplicates()
        else:
            resultado_final = pd.DataFrame(columns = self.df_causal_effects.columns)
            
        return resultado_final
    

    def analyse_explaination(self, sample):
        self.run_dict[sample]['data_analysis'] = []
        for i, content in enumerate(self.run_dict[sample]['list_analyse']):
            self.global_quant_contrafac_max += 1
            controle = {}
            causal = content[0]
            df = content[1]
            ori = content[2]
            
            
            num_changes = len(causal)
            self.run_dict['global_numbers']['global_quant_changes'] += num_changes
            
            num_causal_rules = len(df)
            self.run_dict['global_numbers']['global_quant_causal_rules'] += num_causal_rules
            
            for attr in causal:
                key = attr.column
                if attr.value > ori[key]:
                    controle[key] = 'mais'
                else:
                    controle[key] = 'menos'

            df_temp = df.copy()
            df_temp['from'] = df['from'].map(controle)
            df_temp['to'] = df['to'].map(controle)
            df_temp['causal'] = df_temp.apply(self.verificar_condicoes, axis = 1)
            data_dict = {}

            data_dict['df_respeita_causal'] = df_temp
            data_dict['contrafactual_causal'] = causal
            data_dict['df_causal_effects'] = df
            
            self.run_dict[sample]['data_analysis'].append(data_dict)


            causal_finds = df_temp['causal'].sum()
            self.run_dict['global_numbers']['global_quant_causal_changes'] += causal_finds
            
            # print(f'causal = \n{causal}\n')
            # print(f'original = \n{ori}\n')
            # print(f'df_temp = \n{display(df_temp)}\n')
            
            if len(df_temp) > 0:
                if causal_finds > 0:
                    self.run_dict['global_numbers']['global_quant_causal_contrafac'] += 1
                else:
                    # print(f'nenhuma relaçao causal satisfeita')
                    self.run_dict['global_numbers']['global_quant_zeros_causal'] += 1
    #                 display(df_temp)
    #                 print(f"original = {ori}")
    #                 print(f"causal = {causal}")

                if causal_finds == num_causal_rules:
                    self.run_dict['global_numbers']['global_quant_full_causal'] += 1
                    # if causal_finds > 2:
                        # print(f'todas > 2 relaçoes causais satisfeitas')
    #                     display(df_temp)
    #                     print(f"original = {ori}")
    #                     print(f"causal = {causal}")
                    # elif causal_finds == 1:
                        # print(f'todas = 1 relaçoes causais satisfeitas')
                
                if causal_finds >= (len(df_temp)/2):
                    self.run_dict['global_numbers']['global_quant_maioria_causal_satisfeita'] += 1
            else:
    #             if len(causal) > 0:
                self.run_dict['global_numbers']['global_quant_contrafac_unico'] += 1
        
    def verificar_condicoes(self, row):
        if (row['from'] == 'mais' and row['to'] == 'mais' and row['effect'] > 0):
            return True
        elif row['from'] == 'menos' and row['to'] == 'menos' and row['effect'] > 0:
            return True
        elif row['from'] == 'mais' and row['to'] == 'menos' and row['effect'] < 0:
            return True
        elif row['from'] == 'menos' and row['to'] == 'mais' and row['effect'] < 0:
            return True
        else:
            return False
            

    def show_metrics(self):
        
        print(f"Quantidade de instâncias contrafactuais = {self.global_quant_contrafac_max}")
        print(f'Quantidade de relações causais na base de dados = {len(self.df_causal_effects)}')
        print(f"Quantidade de atributos modificados = {self.run_dict['global_numbers']['global_quant_changes']}")
        print(f"Quantidade de instâncias contrafactuais causais = {self.run_dict['global_numbers']['global_quant_contrafac_unico'] + self.run_dict['global_numbers']['global_quant_causal_contrafac']}")
        print(f"Quantidade de relações causais analisadas = {self.run_dict['global_numbers']['global_quant_causal_rules']}")
        print(f"Quantidade de relações causais satisfeitas = {self.run_dict['global_numbers']['global_quant_causal_changes']}")
        print(f"Quantidade de instâncias contrafactuais com um único atributo modificado = {self.run_dict['global_numbers']['global_quant_contrafac_unico']}")
        
              
              
              
#         print(f"quantas instancias tiveram pelo menos uma relação causal satisfeita = {self.run_dict['global_numbers']['global_quant_causal_contrafac']}/{self.global_quant_contrafac_max}")

#         print(f"quantas relações causais foram encontradas = {self.run_dict['global_numbers']['global_quant_causal_rules']}")

#         print(f"quantas relações causais foram satisfeitas = {self.run_dict['global_numbers']['global_quant_causal_changes']}/{self.run_dict['global_numbers']['global_quant_causal_rules']}")

#         print(f"quantas instâncias não tiveram nenhuma relação causal satisfeita = {self.run_dict['global_numbers']['global_quant_zeros_causal']}/{self.global_quant_contrafac_max}")
#         print(f"quantas instâncias tiveram TODAS as relaçoes causais satisfeitas = {self.run_dict['global_numbers']['global_quant_full_causal']}/{self.run_dict['global_numbers']['global_quant_causal_contrafac']}")

#         print(f"quantas instancias causais teveram a maioria das relacoes causais satisfeitas = {self.run_dict['global_numbers']['global_quant_maioria_causal_satisfeita']}/{self.global_quant_contrafac_max}")
#         print(f"quantas instancias tiveram um unico atributo modificado = {self.run_dict['global_numbers']['global_quant_contrafac_unico']}/{self.global_quant_contrafac_max}")
        print('---------------------------------------------------------------------------------------')

        

# German medium

In [48]:
samples_index = [238, 349, 597, 646, 704, 747, 76, 839, 890, 979]

In [3]:
ccsse = CCSSE('german_medium', samples = 10, K = 1, generation= 10)

              precision    recall  f1-score   support

         0.0       0.46      0.50      0.48        66
         1.0       0.58      0.55      0.56        84

    accuracy                           0.53       150
   macro avg       0.52      0.52      0.52       150
weighted avg       0.53      0.53      0.53       150



In [60]:
ccsse.run_causal()

Sex                          0.0
Age                         37.0
Credit                    1274.0
LoanDuration                12.0
CheckingAccountBalance       1.0
SavingsAccountBalance        1.0
HousingStatus                3.0
Name: 747, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:10<00:00,  1.02s/it]


Sex                          1.0
Age                         32.0
Credit                    1136.0
LoanDuration                 9.0
CheckingAccountBalance       2.0
SavingsAccountBalance        3.0
HousingStatus                1.0
Name: 349, dtype: float64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:10<00:00,  1.05s/it]


Sex                          1.0
Age                         27.0
Credit                    2613.0
LoanDuration                36.0
CheckingAccountBalance       1.0
SavingsAccountBalance        1.0
HousingStatus                3.0
Name: 225, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:10<00:00,  1.09s/it]


In [61]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 11
Quantidade de relações causais na base de dados = 3
Quantidade de atributos modificados = 11
Quantidade de instâncias contrafactuais causais = 10
Quantidade de relações causais analisadas = 1
Quantidade de relações causais satisfeitas = 1.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 9
---------------------------------------------------------------------------------------


In [71]:
ccsse.run_dict

{'global_numbers': {'global_quant_changes': 11,
  'global_quant_causal_changes': 1.0,
  'global_quant_causal_rules': 1,
  'global_quant_zeros_causal': 0,
  'global_quant_full_causal': 1,
  'global_quant_causal_contrafac': 1,
  'global_quant_maioria_causal_satisfeita': 1,
  'global_quant_contrafac_unico': 9},
 0: {'original_instance': Sex                          0.0
  Age                         37.0
  Credit                    1274.0
  LoanDuration                12.0
  CheckingAccountBalance       1.0
  SavingsAccountBalance        1.0
  HousingStatus                3.0
  Name: 747, dtype: float64,
  'causal_explain': [[[('Credit', 11180.972953669236)]],
   Empty DataFrame
   Columns: [from, to, effect]
   Index: [],
   Sex                          0.0
   Age                         37.0
   Credit                    1274.0
   LoanDuration                12.0
   CheckingAccountBalance       1.0
   SavingsAccountBalance        1.0
   HousingStatus                3.0
   Name: 747, dtype

In [73]:
ccsse_10 = CCSSE('german_medium', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

         0.0       0.44      0.48      0.46        66
         1.0       0.56      0.52      0.54        84

    accuracy                           0.51       150
   macro avg       0.50      0.50      0.50       150
weighted avg       0.51      0.51      0.51       150



In [74]:
ccsse_10.run_causal()

Sex                          0.0
Age                         37.0
Credit                    1274.0
LoanDuration                12.0
CheckingAccountBalance       1.0
SavingsAccountBalance        1.0
HousingStatus                3.0
Name: 747, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


Sex                          1.0
Age                         32.0
Credit                    1136.0
LoanDuration                 9.0
CheckingAccountBalance       2.0
SavingsAccountBalance        3.0
HousingStatus                1.0
Name: 349, dtype: float64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:10<00:00,  1.08s/it]


Sex                          1.0
Age                         27.0
Credit                    2613.0
LoanDuration                36.0
CheckingAccountBalance       1.0
SavingsAccountBalance        1.0
HousingStatus                3.0
Name: 225, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:11<00:00,  1.13s/it]


In [75]:
ccsse_10.show_metrics()

Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 3
Quantidade de atributos modificados = 76
Quantidade de instâncias contrafactuais causais = 45
Quantidade de relações causais analisadas = 16
Quantidade de relações causais satisfeitas = 12.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 34
---------------------------------------------------------------------------------------


# German short

In [65]:
ccsse_short = CCSSE('german_short', samples = 10, K = 1, generation= 10)

              precision    recall  f1-score   support

         0.0       0.44      0.62      0.52        66
         1.0       0.56      0.38      0.45        84

    accuracy                           0.49       150
   macro avg       0.50      0.50      0.48       150
weighted avg       0.51      0.49      0.48       150



In [66]:
ccsse_short.run_causal()

Sex                0.0
Age               37.0
Credit          1274.0
LoanDuration      12.0
Name: 747, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:11<00:00,  1.12s/it]


Sex                1.0
Age               32.0
Credit          1136.0
LoanDuration       9.0
Name: 349, dtype: float64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:11<00:00,  1.12s/it]


Sex                1.0
Age               27.0
Credit          2613.0
LoanDuration      36.0
Name: 225, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:10<00:00,  1.08s/it]


In [67]:
ccsse_short.show_metrics()

Quantidade de instâncias contrafactuais = 11
Quantidade de relações causais na base de dados = 2
Quantidade de atributos modificados = 10
Quantidade de instâncias contrafactuais causais = 10
Quantidade de relações causais analisadas = 0
Quantidade de relações causais satisfeitas = 0.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 10
---------------------------------------------------------------------------------------


In [72]:
ccsse_short.run_dict

{'global_numbers': {'global_quant_changes': 10,
  'global_quant_causal_changes': 0.0,
  'global_quant_causal_rules': 0,
  'global_quant_zeros_causal': 0,
  'global_quant_full_causal': 0,
  'global_quant_causal_contrafac': 0,
  'global_quant_maioria_causal_satisfeita': 0,
  'global_quant_contrafac_unico': 10},
 0: {'original_instance': Sex                0.0
  Age               37.0
  Credit          1274.0
  LoanDuration      12.0
  Name: 747, dtype: float64,
  'causal_explain': [[[('Credit', 1504.665492590474)]],
   Empty DataFrame
   Columns: [from, to, effect]
   Index: [],
   Sex                0.0
   Age               37.0
   Credit          1274.0
   LoanDuration      12.0
   Name: 747, dtype: float64],
  'list_analyse': [[[('Credit', 1504.665492590474)],
    Empty DataFrame
    Columns: [from, to, effect]
    Index: [],
    Credit    1274.0
    Name: 747, dtype: float64]],
  'data_analysis': [{'df_respeita_causal': Empty DataFrame
    Columns: [from, to, effect, causal]
    Inde

In [68]:
ccsse_short10 = CCSSE('german_short', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

         0.0       0.44      0.61      0.51        66
         1.0       0.57      0.40      0.47        84

    accuracy                           0.49       150
   macro avg       0.51      0.51      0.49       150
weighted avg       0.51      0.49      0.49       150



In [69]:
ccsse_short10.run_causal()

Sex                0.0
Age               37.0
Credit          1274.0
LoanDuration      12.0
Name: 747, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:11<00:00,  1.12s/it]


Sex                1.0
Age               32.0
Credit          1136.0
LoanDuration       9.0
Name: 349, dtype: float64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:10<00:00,  1.06s/it]


Sex                1.0
Age               27.0
Credit          2613.0
LoanDuration      36.0
Name: 225, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:10<00:00,  1.09s/it]


In [70]:
ccsse_short10.show_metrics()

Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 2
Quantidade de atributos modificados = 38
Quantidade de instâncias contrafactuais causais = 25
Quantidade de relações causais analisadas = 6
Quantidade de relações causais satisfeitas = 3.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 22
---------------------------------------------------------------------------------------


In [79]:
ccsse_short10.run_dict

{'global_numbers': {'global_quant_changes': 38,
  'global_quant_causal_changes': 3.0,
  'global_quant_causal_rules': 6,
  'global_quant_zeros_causal': 3,
  'global_quant_full_causal': 3,
  'global_quant_causal_contrafac': 3,
  'global_quant_maioria_causal_satisfeita': 3,
  'global_quant_contrafac_unico': 22},
 0: {'original_instance': Sex                0.0
  Age               37.0
  Credit          1274.0
  LoanDuration      12.0
  Name: 747, dtype: float64,
  'causal_explain': [[[('Credit', 1016.8134710586951)],
    [('Age', 41.251004095010046)]],
   Empty DataFrame
   Columns: [from, to, effect]
   Index: [],
   Sex                0.0
   Age               37.0
   Credit          1274.0
   LoanDuration      12.0
   Name: 747, dtype: float64],
  'list_analyse': [[[('Credit', 1016.8134710586951)],
    Empty DataFrame
    Columns: [from, to, effect]
    Index: [],
    Credit    1274.0
    Name: 747, dtype: float64],
   [[('Age', 41.251004095010046)],
    Empty DataFrame
    Columns: [fr

# ----------------------------------------------------------------------------------

# Breast Coimbra np

In [22]:
ccsse = CCSSE('np', samples = 10, K = 1, generation= 10)

              precision    recall  f1-score   support

           1       0.80      0.80      0.80         5
           2       0.86      0.86      0.86         7

    accuracy                           0.83        12
   macro avg       0.83      0.83      0.83        12
weighted avg       0.83      0.83      0.83        12



In [23]:
ccsse.run_causal()

Age             61.000000
BMI             32.038959
Glucose         85.000000
Insulin         18.077000
HOMA             3.790144
Leptin          30.772900
Adiponectin      7.780255
Resistin        13.683920
MCP.1          444.395000
Name: 17, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


Age             58.000000
BMI             29.154519
Glucose        139.000000
Insulin         16.582000
HOMA             5.685415
Leptin          22.888400
Adiponectin     10.262660
Resistin        13.973990
MCP.1          923.886000
Name: 89, dtype: float64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:09<00:00,  1.07it/s]


Age             66.000000
BMI             31.238590
Glucose         82.000000
Insulin          4.181000
HOMA             0.845677
Leptin          16.224700
Adiponectin      4.267105
Resistin         3.291750
MCP.1          634.602000
Name: 36, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:11<00:00,  1.10s/it]


In [24]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 11
Quantidade de relações causais na base de dados = 12
Quantidade de atributos modificados = 16
Quantidade de instâncias contrafactuais causais = 8
Quantidade de relações causais analisadas = 4
Quantidade de relações causais satisfeitas = 2.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 6
---------------------------------------------------------------------------------------


# ----------------------------------------------------------------------------------

# Heloc nm

In [3]:
ccsse = CCSSE('nm', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

         Bad       0.74      0.80      0.77       563
        Good       0.74      0.67      0.70       483

    accuracy                           0.74      1046
   macro avg       0.74      0.73      0.73      1046
weighted avg       0.74      0.74      0.74      1046



In [4]:
ccsse.run_causal()

ExternalRiskEstimate                   64
MSinceOldestTradeOpen                 345
MSinceMostRecentTradeOpen               3
AverageMInFile                        109
NumSatisfactoryTrades                  31
NumTrades60Ever2DerogPubRec             2
NumTrades90Ever2DerogPubRec             1
PercentTradesNeverDelq                 92
MSinceMostRecentDelq                   59
MaxDelq2PublicRecLast12M                6
MaxDelqEver                             2
NumTotalTrades                         36
NumTradesOpeninLast12M                  3
PercentInstallTrades                   42
MSinceMostRecentInqexcl7days            0
NumInqLast6M                            3
NumInqLast6Mexcl7days                   3
NetFractionRevolvingBurden             51
NetFractionInstallBurden               78
NumRevolvingTradesWBalance              6
NumInstallTradesWBalance                1
NumBank2NatlTradesWHighUtilization      3
PercentTradesWBalance                  64
Name: 350, dtype: int64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:53<00:00,  5.30s/it]


ExternalRiskEstimate                   75
MSinceOldestTradeOpen                 310
MSinceMostRecentTradeOpen               1
AverageMInFile                         83
NumSatisfactoryTrades                  25
NumTrades60Ever2DerogPubRec             0
NumTrades90Ever2DerogPubRec             0
PercentTradesNeverDelq                100
MSinceMostRecentDelq                   -7
MaxDelq2PublicRecLast12M                7
MaxDelqEver                             8
NumTotalTrades                         13
NumTradesOpeninLast12M                  4
PercentInstallTrades                   32
MSinceMostRecentInqexcl7days           -7
NumInqLast6M                            1
NumInqLast6Mexcl7days                   1
NetFractionRevolvingBurden             28
NetFractionInstallBurden               83
NumRevolvingTradesWBalance              3
NumInstallTradesWBalance                4
NumBank2NatlTradesWHighUtilization      1
PercentTradesWBalance                  62
Name: 2806, dtype: int64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:30<00:00,  3.02s/it]


ExternalRiskEstimate                   76
MSinceOldestTradeOpen                 356
MSinceMostRecentTradeOpen               2
AverageMInFile                         56
NumSatisfactoryTrades                  34
NumTrades60Ever2DerogPubRec             0
NumTrades90Ever2DerogPubRec             0
PercentTradesNeverDelq                100
MSinceMostRecentDelq                   -7
MaxDelq2PublicRecLast12M                7
MaxDelqEver                             8
NumTotalTrades                         29
NumTradesOpeninLast12M                  6
PercentInstallTrades                   22
MSinceMostRecentInqexcl7days            0
NumInqLast6M                            6
NumInqLast6Mexcl7days                   6
NetFractionRevolvingBurden              2
NetFractionInstallBurden               66
NumRevolvingTradesWBalance              4
NumInstallTradesWBalance                3
NumBank2NatlTradesWHighUtilization      0
PercentTradesWBalance                  39
Name: 3580, dtype: int64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:34<00:00,  3.47s/it]


In [5]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 170
Quantidade de atributos modificados = 87
Quantidade de instâncias contrafactuais causais = 20
Quantidade de relações causais analisadas = 168
Quantidade de relações causais satisfeitas = 116.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 1
---------------------------------------------------------------------------------------


# ----------------------------------------------------------------------------------

# tokyo ng

In [25]:
ccsse = CCSSE('tokyo_ng', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

           0       0.90      0.88      0.89        40
           1       0.91      0.93      0.92        56

    accuracy                           0.91        96
   macro avg       0.90      0.90      0.90        96
weighted avg       0.91      0.91      0.91        96



In [26]:
ccsse.run_causal()

cpu_avg_user            0.002021
cpu_avg_sys             0.012630
cpu_avg_busy            0.014650
cpu_avg_wait            0.000146
cpu_avg_idle            0.981500
cpu_avg_waste           0.981600
cpu_max_user            0.003833
cpu_max_sys             0.027500
cpu_max_busy            0.030670
cpu_max_wait            0.000167
cpu_max_idle            0.994700
cpu_max_waste           0.994900
cpu_frac_busy           0.000000
io_iget              3004.000000
io_bread             7646.000000
io_bwrite           24490.000000
io_lread            20800.000000
io_lwrite               0.000000
io_phread               0.000000
io_phwrite            750.900000
io_wcancel              3.600000
io_namei                0.000000
io_dirblk               0.001071
disk_avg_active         0.012850
disk_max_active         0.000000
disk_frac_active        0.000000
disk_avg_read           0.077780
disk_avg_write          0.077780
disk_avg_total          0.000000
disk_max_read           0.933300
disk_max_w

Running original instance:
 None


Processing...:   0%|          | 0/10 [00:08<?, ?it/s]


ValueError: array must not contain infs or NaNs

In [ ]:
ccsse.show_metrics()

# ----------------------------------------------------------------------------------

# Breast Cancer ng

In [3]:
ccsse = CCSSE('breast-cancer_ng', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

           B       0.94      0.97      0.96        34
           M       0.95      0.91      0.93        23

    accuracy                           0.95        57
   macro avg       0.95      0.94      0.94        57
weighted avg       0.95      0.95      0.95        57



In [4]:
ccsse.run_causal()

id                         8.974280e+07
radius_mean                1.706000e+01
texture_mean               2.100000e+01
perimeter_mean             1.118000e+02
area_mean                  9.186000e+02
smoothness_mean            1.119000e-01
compactness_mean           1.056000e-01
concavity_mean             1.508000e-01
concave points_mean        9.934000e-02
symmetry_mean              1.727000e-01
fractal_dimension_mean     6.071000e-02
radius_se                  8.161000e-01
texture_se                 2.129000e+00
perimeter_se               6.076000e+00
area_se                    8.717000e+01
smoothness_se              6.455000e-03
compactness_se             1.797000e-02
concavity_se               4.502000e-02
concave points_se          1.744000e-02
symmetry_se                1.829000e-02
fractal_dimension_se       3.733000e-03
radius_worst               2.099000e+01
texture_worst              3.315000e+01
perimeter_worst            1.432000e+02
area_worst                 1.362000e+03


Running original instance:
 None


Processing...: 100%|██████████| 10/10 [01:49<00:00, 10.91s/it]


id                         886452.000000
radius_mean                    13.960000
texture_mean                   17.050000
perimeter_mean                 91.430000
area_mean                     602.400000
smoothness_mean                 0.109600
compactness_mean                0.127900
concavity_mean                  0.097890
concave points_mean             0.052460
symmetry_mean                   0.190800
fractal_dimension_mean          0.061300
radius_se                       0.425000
texture_se                      0.809800
perimeter_se                    2.563000
area_se                        35.740000
smoothness_se                   0.006351
compactness_se                  0.026790
concavity_se                    0.031190
concave points_se               0.013420
symmetry_se                     0.020620
fractal_dimension_se            0.002695
radius_worst                   16.390000
texture_worst                  22.070000
perimeter_worst               108.100000
area_worst      

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [01:54<00:00, 11.47s/it]


id                         909411.000000
radius_mean                    10.970000
texture_mean                   17.200000
perimeter_mean                 71.730000
area_mean                     371.500000
smoothness_mean                 0.089150
compactness_mean                0.111300
concavity_mean                  0.094570
concave points_mean             0.036130
symmetry_mean                   0.148900
fractal_dimension_mean          0.066400
radius_se                       0.257400
texture_se                      1.376000
perimeter_se                    2.806000
area_se                        18.150000
smoothness_se                   0.008565
compactness_se                  0.046380
concavity_se                    0.064300
concave points_se               0.017680
symmetry_se                     0.015160
fractal_dimension_se            0.004976
radius_worst                   12.360000
texture_worst                  26.870000
perimeter_worst                90.140000
area_worst      

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:38<00:00,  3.82s/it]


In [5]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 260
Quantidade de atributos modificados = 146
Quantidade de instâncias contrafactuais causais = 13
Quantidade de relações causais analisadas = 646
Quantidade de relações causais satisfeitas = 375
Quantidade de instâncias contrafactuais com um único atributo modificado = 0
---------------------------------------------------------------------------------------


# ----------------------------------------------------------------------------------

# Monks cp

In [15]:
ccsse = CCSSE('cp', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       216
           1       0.96      0.94      0.95       216

    accuracy                           0.95       432
   macro avg       0.95      0.95      0.95       432
weighted avg       0.95      0.95      0.95       432



In [16]:
ccsse.run_causal()

a1    1
a2    1
a3    1
a4    1
a5    1
a6    1
Name: 0, dtype: int64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:00<00:00, 17.91it/s]


a1    1
a2    1
a3    1
a4    1
a5    1
a6    2
Name: 1, dtype: int64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:00<00:00, 25.81it/s]


a1    1
a2    1
a3    1
a4    2
a5    1
a6    2
Name: 9, dtype: int64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:00<00:00, 25.10it/s]


In [17]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 0
Quantidade de atributos modificados = 64
Quantidade de instâncias contrafactuais causais = 30
Quantidade de relações causais analisadas = 0
Quantidade de relações causais satisfeitas = 0.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 30
---------------------------------------------------------------------------------------


# ----------------------------------------------------------------------------------

# House Votes cm

In [11]:
ccsse = CCSSE('cm', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

    democrat       1.00      1.00      1.00        23
  republican       1.00      1.00      1.00        21

    accuracy                           1.00        44
   macro avg       1.00      1.00      1.00        44
weighted avg       1.00      1.00      1.00        44



In [12]:
ccsse.run_causal()

handicapped-infants                       0
water-project-cost-sharing                1
adoption-of-the-budget-resolution         0
physician-fee-freeze                      1
el-salvador-aid                           1
religious-groups-in-schools               1
anti-satellite-test-ban                   0
aid-to-nicaraguan-contras                 0
mx-missile                                0
immigration                               1
synfuels-corporation-cutback              0
education-spending                        1
superfund-right-to-sue                    1
crime                                     1
duty-free-exports                         0
export-administration-act-south-africa    0
Name: 434, dtype: int64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:04<00:00,  2.01it/s]


handicapped-infants                       0
water-project-cost-sharing                0
adoption-of-the-budget-resolution         1
physician-fee-freeze                      1
el-salvador-aid                           1
religious-groups-in-schools               1
anti-satellite-test-ban                   1
aid-to-nicaraguan-contras                 0
mx-missile                                0
immigration                               1
synfuels-corporation-cutback              1
education-spending                        1
superfund-right-to-sue                    1
crime                                     1
duty-free-exports                         0
export-administration-act-south-africa    1
Name: 353, dtype: int64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:04<00:00,  2.03it/s]


handicapped-infants                       1
water-project-cost-sharing                0
adoption-of-the-budget-resolution         1
physician-fee-freeze                      0
el-salvador-aid                           0
religious-groups-in-schools               1
anti-satellite-test-ban                   1
aid-to-nicaraguan-contras                 1
mx-missile                                1
immigration                               1
synfuels-corporation-cutback              0
education-spending                        0
superfund-right-to-sue                    1
crime                                     0
duty-free-exports                         0
export-administration-act-south-africa    1
Name: 360, dtype: int64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:05<00:00,  1.98it/s]


In [13]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 60
Quantidade de atributos modificados = 321
Quantidade de instâncias contrafactuais causais = 36
Quantidade de relações causais analisadas = 828
Quantidade de relações causais satisfeitas = 638
Quantidade de instâncias contrafactuais com um único atributo modificado = 0
---------------------------------------------------------------------------------------


# ----------------------------------------------------------------------------------

# Mushrooms cg

In [ ]:
ccsse = CCSSE('cg', samples = 10, K = 10, generation= 10)

In [ ]:
ccsse.run_causal()

In [ ]:
ccsse.show_metrics()

# ----------------------------------------------------------------------------------

# German short

In [12]:
ccsse = CCSSE('german_short', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

         0.0       0.46      0.64      0.54        66
         1.0       0.59      0.42      0.49        84

    accuracy                           0.51       150
   macro avg       0.53      0.53      0.51       150
weighted avg       0.54      0.51      0.51       150



In [14]:
ccsse.x_test.columns

Index(['Sex', 'Age', 'Credit', 'LoanDuration'], dtype='object')

In [19]:
ccsse.run_causal()

Sex                0.0
Age               37.0
Credit          1274.0
LoanDuration      12.0
Name: 747, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


Sex                1.0
Age               32.0
Credit          1136.0
LoanDuration       9.0
Name: 349, dtype: float64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:09<00:00,  1.01it/s]


Sex                1.0
Age               27.0
Credit          2613.0
LoanDuration      36.0
Name: 225, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:09<00:00,  1.00it/s]


In [20]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 2
Quantidade de atributos modificados = 37
Quantidade de instâncias contrafactuais causais = 25
Quantidade de relações causais analisadas = 6
Quantidade de relações causais satisfeitas = 4.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 21
---------------------------------------------------------------------------------------


# ----------------------------------------------------------------------------------

# German full

In [15]:
ccsse = CCSSE('german_full', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

           0       0.82      0.92      0.87        72
           1       0.70      0.50      0.58        28

    accuracy                           0.80       100
   macro avg       0.76      0.71      0.73       100
weighted avg       0.79      0.80      0.79       100



In [16]:
len(ccsse.x_test.columns)

20

In [22]:
ccsse.run_causal()

account_check_status             1
duration_in_month                6
credit_history                   2
purpose                          0
credit_amount                 1203
savings                          2
present_emp_since                5
installment_as_income_perc       3
personal_status_sex              3
other_debtors                    1
present_res_since                2
property                         2
age                             43
other_installment_plans          1
housing                          2
credits_this_2                   1
job                              3
people_under_maintenance         1
telephone                        2
foreign_worker                   1
Name: 518, dtype: int64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:16<00:00,  1.70s/it]


account_check_status            1
duration_in_month               6
credit_history                  2
purpose                         0
credit_amount                 662
savings                         1
present_emp_since               2
installment_as_income_perc      3
personal_status_sex             3
other_debtors                   1
present_res_since               4
property                        1
age                            41
other_installment_plans         1
housing                         2
credits_this_2                  1
job                             2
people_under_maintenance        2
telephone                       2
foreign_worker                  1
Name: 529, dtype: int64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:15<00:00,  1.55s/it]


account_check_status             3
duration_in_month               15
credit_history                   2
purpose                          4
credit_amount                 2327
savings                          1
present_emp_since                2
installment_as_income_perc       2
personal_status_sex              2
other_debtors                    1
present_res_since                3
property                         1
age                             25
other_installment_plans          1
housing                          2
credits_this_2                   1
job                              2
people_under_maintenance         1
telephone                        1
foreign_worker                   1
Name: 834, dtype: int64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:21<00:00,  2.15s/it]


In [23]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 59
Quantidade de atributos modificados = 135
Quantidade de instâncias contrafactuais causais = 29
Quantidade de relações causais analisadas = 186
Quantidade de relações causais satisfeitas = 150.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 7
---------------------------------------------------------------------------------------


# ----------------------------------------------------------------------------------

# Phishing_cg

In [8]:
ccsse = CCSSE('Phishing_cg', samples = 10, K = 10, generation= 10)

              precision    recall  f1-score   support

         0.0       0.92      0.96      0.94        49
         1.0       0.96      0.92      0.94        51

    accuracy                           0.94       100
   macro avg       0.94      0.94      0.94       100
weighted avg       0.94      0.94      0.94       100



In [9]:
ccsse.df_causal_effectsausal_effects

,from,to,effect
0,double_slash_redirecting,having_IP_Address,0.704897
1,Redirect,having_IP_Address,-0.182756
2,having_IP_Address,URL_Length,0.482936
3,Shortining_Service,URL_Length,-0.496544
4,port,URL_Length,0.161871
...,...,...,...
99,Submitting_to_email,Statistical_report,0.186762
100,Abnormal_URL,Statistical_report,0.671463
101,Redirect,Statistical_report,0.343433
102,on_mouseover,Statistical_report,0.098306


In [10]:
ccsse.run_causal()

having_IP_Address               1.0
URL_Length                      0.0
Shortining_Service              1.0
having_At_Symbol                1.0
double_slash_redirecting        1.0
Prefix_Suffix                   0.0
having_Sub_Domain               0.0
SSLfinal_State                  2.0
Domain_registeration_length     1.0
Favicon                         1.0
port                            1.0
HTTPS_token                     1.0
Request_URL                     1.0
URL_of_Anchor                   1.0
Links_in_tags                   1.0
SFH                             0.0
Submitting_to_email             1.0
Abnormal_URL                    1.0
Redirect                        0.0
on_mouseover                    1.0
RightClick                      1.0
popUpWidnow                     1.0
Iframe                          1.0
age_of_domain                   1.0
DNSRecord                       0.0
web_traffic                     0.0
Page_Rank                       0.0
Google_Index                

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:19<00:00,  1.96s/it]


having_IP_Address               1.0
URL_Length                      0.0
Shortining_Service              1.0
having_At_Symbol                1.0
double_slash_redirecting        1.0
Prefix_Suffix                   0.0
having_Sub_Domain               2.0
SSLfinal_State                  2.0
Domain_registeration_length     0.0
Favicon                         1.0
port                            1.0
HTTPS_token                     1.0
Request_URL                     1.0
URL_of_Anchor                   1.0
Links_in_tags                   0.0
SFH                             0.0
Submitting_to_email             1.0
Abnormal_URL                    1.0
Redirect                        0.0
on_mouseover                    1.0
RightClick                      1.0
popUpWidnow                     1.0
Iframe                          1.0
age_of_domain                   1.0
DNSRecord                       1.0
web_traffic                     2.0
Page_Rank                       1.0
Google_Index                

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:26<00:00,  2.61s/it]


having_IP_Address               1.0
URL_Length                      2.0
Shortining_Service              1.0
having_At_Symbol                0.0
double_slash_redirecting        1.0
Prefix_Suffix                   0.0
having_Sub_Domain               1.0
SSLfinal_State                  1.0
Domain_registeration_length     1.0
Favicon                         1.0
port                            1.0
HTTPS_token                     1.0
Request_URL                     0.0
URL_of_Anchor                   0.0
Links_in_tags                   1.0
SFH                             2.0
Submitting_to_email             1.0
Abnormal_URL                    1.0
Redirect                        0.0
on_mouseover                    1.0
RightClick                      1.0
popUpWidnow                     1.0
Iframe                          1.0
age_of_domain                   1.0
DNSRecord                       1.0
web_traffic                     0.0
Page_Rank                       0.0
Google_Index                

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:26<00:00,  2.61s/it]


In [11]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 104
Quantidade de atributos modificados = 42
Quantidade de instâncias contrafactuais causais = 13
Quantidade de relações causais analisadas = 23
Quantidade de relações causais satisfeitas = 20.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 2
---------------------------------------------------------------------------------------


# ----------------------------------------------------------------------------------